### MNIST Classifier using a pre-trained ResNet model

In this exercise, we use a ResNet-18 model to classify items from the MNIST dataset.

First, we import the necessary modules:

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

#### Load pre-trained model

We use a pre-trained ResNet-18 model loaded using `torchvision.models`. Since the ResNet model's head is connected to ImageNet-1k, we remove this layer with the goal to replace it with MNIST.

In [2]:
# Load the Pre-trained ResNet-18 Model
resnet18 = torchvision.models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features

# Remove the fully connected layer (head)
resnet18.fc = nn.Identity()

c:\Users\Carlo Alamani\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Carlo Alamani\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


#### Dataloaders

We create the train and test dataloaders by using `torchvision.datasets` to download the MNIST dataset. Some augmentation is also applied to the dataset through `torchvision.transforms`. Note that transformation is required to match the MNIST channels to that of the ResNet model. Because of this, the dataset is normalized and resized.

In [3]:
# Create the MNIST Dataloader
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match ResNet input size
    transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels to match ResNet input
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST mean and std deviation
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#### Modifying the ResNet model for MNIST

The base ResNet18 model is further modified by configuring the head (fully connected layer) to have 10 classes, matching the MNIST dataset. The tail is freezed in training, leaving only the fine tuned head.

In [4]:
# Modify the ResNet-18 Model for MNIST
class MNIST_ResNet(nn.Module):
    def __init__(self, resnet):
        super(MNIST_ResNet, self).__init__()
        self.resnet = resnet
        self.fc = nn.Linear(num_ftrs, 10)  # 10 classes in MNIST

    def forward(self, x):
        x = self.resnet(x)
        x = self.fc(x)
        return x

model = MNIST_ResNet(resnet18)

# Freeze the Tail and Fine-tune the Head
for param in model.resnet.parameters():
    param.requires_grad = False

# Only train the fully connected layer
for param in model.fc.parameters():
    param.requires_grad = True

#### Training Loop

The modified model is trained for 10 epochs, with the accuracies shown.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
lr = 0.001
num_epochs = 10

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    train_loss = running_loss / len(train_loader)
    train_accuracy = 100. * correct / total
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

#### Evaluation



In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating", leave=False):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_accuracy = 100. * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")